# Trigger finger UKB genomic QC

## Load Hail instance

In [ ]:
import seaborn as sns
import hail as hl
import os
from gnomad.utils.liftover import *
from gnomad.utils.annotations import *
from gnomad.sample_qc.pipeline import *
from gnomad.sample_qc.ancestry import *

tmp = "/mnt/grid/janowitz/home/skleeman/tmp2"
os.environ["SPARK_LOCAL_DIRS"]=tmp

os.environ["PYSPARK_SUBMIT_ARGS"] ="--driver-memory 200g --executor-memory 2g pyspark-shell"

hl.init(default_reference='GRCh38', master='local[16]',min_block_size=128, local_tmpdir=tmp, tmp_dir=tmp)


## Merge UK biobank PLINK data

In [ ]:
#Shell script

plink \
  --bed /mnt/grid/ukbiobank/data/genotypes/plink/ukb_cal_chr1_v2.bed \
  --bim /mnt/grid/janowitz/home/skleeman/ukbiobank/rawdata/ukb_snp_chr1_v2.bim \
  --fam /mnt/grid/ukbiobank/data/ApplicationXXXXX/rawdata/ukbXXXXX_cal_chr1_v2_s488250.fam \
  --merge-list ~/list_beds.txt \
  --make-bed --out ukb_merged \

In [ ]:
bed = '/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/merged_plink/ukb_merged.bed'
bim = '/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/merged_plink/ukb_merged.bim'
fam = '/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/merged_plink/ukb_merged.fam'

ukb = hl.import_plink(bed = bed, bim = bim, fam = fam, reference_genome='GRCh37',
                     min_partitions = 150)

ukb.write('/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/ukb_merged_raw.mt', overwrite=True) #Save raw data

## UKB pre-process

### Filter - GWAS

MAF > 1% (I checked none are >99%), missing rate < 5%

This will form the input to BOLT-LMM

In [ ]:
#Import UKB data in PLINK form

ukb = hl.read_matrix_table('/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/ukb_merged_raw.mt')

print(ukb.count()) #How many variants to start with?

#Filtering - based partly on https://github.com/Nealelab/UK_Biobank_GWAS/blob/master/imputed-v2-gwas/README.md

ukb = hl.variant_qc(ukb) #Default Hail variant QC pipeline
ukb = ukb.filter_rows(ukb.variant_qc.AF[1] > 0.01) #MAF > 1%
ukb = ukb.filter_rows(ukb.variant_qc.call_rate > 0.95) #Filter by call rate > 95%

print(ukb.count()) #How many variants are left?

ukb.write('/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/ukb_grch37_filtered_forgwas.mt', overwrite=True) #Save pruned MT

In [ ]:
ukb = hl.read_matrix_table('/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/ukb_merged_raw.mt')

print(ukb.count()) #How many variants to start with?

#Filtering - based partly on https://github.com/Nealelab/UK_Biobank_GWAS/blob/master/imputed-v2-gwas/README.md

ukb = hl.variant_qc(ukb) #Default Hail variant QC pipeline
ukb = ukb.filter_rows(ukb.variant_qc.AF[1] > 0.99) #MAF > 1%

print(ukb.count()) #How many variants are left?


## Ancestry analysis, sample QC

### Merge UKB and Reference cohorts

Biallelic SNPs, lift to grch38, remove allele mismatches, set AF > 0.1% and call rate >99% in UKB, identify high quality reference variants in WGS data (call rate >95%, biallelic, SNPs cf. indels).

In [ ]:
import hail as hl
import os
from gnomad.utils.liftover import *
from gnomad.utils.annotations import *
from gnomad.sample_qc.pipeline import *
import subprocess

#Define memory and CPU availability

tmp = "/mnt/grid/janowitz/home/skleeman/tmp"

os.environ["SPARK_LOCAL_DIRS"]=tmp
os.environ["PYSPARK_SUBMIT_ARGS"] ="--master local[96] --driver-memory 960G pyspark-shell"


hl.init(default_reference='GRCh38', master ='local[96]', local='local[96]',min_block_size=128, local_tmpdir=tmp, tmp_dir=tmp)


#Import UKB
ukb = hl.read_matrix_table('/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/ukb_merged_raw.mt')
print(ukb.count()) #How many variants to start with?
ukb = default_lift_data(ukb) #GNOMAD pipeline for liftover, including reverse complement on negative strand

#Filtering

ukb = hl.variant_qc(ukb) #Default Hail variant QC pipeline
ukb = ukb.filter_rows(hl.len(ukb.alleles) == 2) #Biallelic SNPs only
ukb = ukb.filter_rows(ukb.ref_allele_mismatch == False) #Remove alleles with reference mismatch ('allele flips')
ukb = ukb.filter_rows(ukb.variant_qc.AF[1] > 0.001) #MAF > 0.1%
ukb = ukb.filter_rows(ukb.variant_qc.call_rate > 0.99) #Filter by call rate > 99%
ukb = ukb.checkpoint('/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/ukb_grch38_filtered.mt', overwrite=True) #Save filtered MT

#Exclude LD intervals from plinkQC package, LD pruning in PLINK (not working in Hail due to bug)
intervals = hl.import_bed('/mnt/grid/janowitz/home/skleeman/ukbiobank/cancergwas/remove_ld_grch38.bed',
                         reference_genome='GRCh38')
ukb = ukb.filter_rows(hl.is_defined(intervals[ukb.locus]),keep=False)

hl.export_plink(ukb, '/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/pre_ld_plink',
                fam_id = ukb.fam_id, ind_id = ukb.s, pat_id = ukb.pat_id, mat_id = ukb.mat_id,
                is_female = ukb.is_female)

commands = '''
plink --bfile /mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/pre_ld_plink \
--indep-pairwise 50 5 0.2 --out /mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/prune --threads 96 --memory 50000 \
--allow-extra-chr

plink --bfile /mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/pre_ld_plink \
--extract /mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/prune.prune.in \
--make-bed --out /mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/post_ld_plink --threads 96 --memory 50000 \
--allow-extra-chr
'''

output = subprocess.check_output(commands, shell=True)
print(output)

bed = '/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/post_ld_plink.bed'
bim = '/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/post_ld_plink.bim'
fam = '/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/post_ld_plink.fam'

ukb = hl.import_plink(bed = bed, bim = bim, fam = fam, reference_genome='GRCh38',
                     min_partitions = 150)
print(ukb.count())
ukb = ukb.checkpoint('/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/ukb_grch38_pruned.mt', overwrite=True) #Save filtered MT

#Import REF
this_ref = hl.read_matrix_table('/mnt/grid/janowitz/rdata_norepl/ref.mt') #Dense form (!!!)

this_ref = hl.variant_qc(this_ref) #Default Hail variant QC pipeline
this_ref = this_ref.filter_rows((hl.len(this_ref.alleles) == 2) & hl.is_snp(this_ref.alleles[0], this_ref.alleles[1]))
this_ref = this_ref.filter_rows(this_ref.variant_qc.call_rate > 0.95) #Filter by call rate > 95%
this_ref = this_ref.naive_coalesce(250)

this_ref = this_ref.checkpoint('/mnt/grid/janowitz/home/references/1k_hgdp/ref_gnomadfilters.mt', overwrite=True) #Save filtered MT

#Intersect
ukbb_in_ref = ukb.filter_rows(hl.is_defined(this_ref.rows()[ukb.row_key]))
print('sites in ref and UKBB data, inds in UKBB: ' + str(ukbb_in_ref.count()))

ref_in_ukbb = this_ref.filter_rows(hl.is_defined(ukb.rows()[this_ref.row_key]))
print('sites in ref and UKBB data, inds in ref: ' + str(ref_in_ukbb.count()))

#Export
ref_in_ukbb.write('/mnt/grid/janowitz/home/references/1k_hgdp/ref_intersect.mt', overwrite=True)
ukbb_in_ref.write('/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/ukb_grch38_intersect.mt',overwrite=True)

plinkQC reference

In [ ]:
this_ref = hl.read_matrix_table('/mnt/grid/janowitz/home/references/1k_hgdp/ref_intersect.mt')
this_ref = default_lift_data(this_ref) 

related_samples_to_remove_ref = hl.read_table("/mnt/grid/janowitz/home/references/1k_hgdp/related_remove_ref.ht")

this_ref = this_ref.filter_cols(hl.is_defined(related_samples_to_remove_ref[this_ref.col_key]), keep=False)

this_ref.count()

In [ ]:
this_ref = hl.read_matrix_table('/mnt/grid/janowitz/home/references/1k_hgdp/ref_intersect.mt') #Dense form (!!!)
snps = this_ref.rows()
snps = snps.select_globals()
snps = snps.select()
snps.write('/mnt/grid/janowitz/home/references/1k_hgdp/ukb_snp_reference.ht',overwrite=True)

### Run PCA and RF classifier

Train classifier on reference set, pre-process with removal of related samples to 3rd degree (PC-Relate threshold of 0.05), extract 10 principal components, train RF classifier using populations labels subdivided per https://docs.google.com/spreadsheets/d/1jenSz_HnbA1kBESaUmur3Ob72-EPXJgfUWhbz5UdltA/edit#gid=433808438. Define populations using RF classifer probability threshold of 0.70.

Run on cluster

In [ ]:
import pickle
import pandas as pd
from gnomad.sample_qc.ancestry import *



this_ref = hl.read_matrix_table('/mnt/grid/janowitz/home/references/1k_hgdp/ref_intersect.mt')
ukb = hl.read_matrix_table('/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/ukb_grch38_intersect.mt')


#Find related individuals in 1000K/HGDP set

relatedness_ht = hl.pc_relate(this_ref.GT, 0.01, k=10, min_kinship=0.05, block_size=512)

related_samples_to_remove_ref = hl.maximal_independent_set(relatedness_ht.i, relatedness_ht.j, False)


#Run PCA

#--> Reference, label with inferred populations, exclude relateds
_, scores_pca_ref, loadings_pca_ref = run_pca_with_relateds(this_ref, related_samples_to_remove_ref, 
                                                               n_pcs=10, autosomes_only=True)

#--> UKB

scores_pca_ukb = ancestry.pc_project(mt = ukb, loadings_ht = loadings_pca_ref)

scores_pca_ref.write("/mnt/grid/janowitz/home/references/1k_hgdp/scores_pca_ref.ht", overwrite=True)
scores_pca_ukb.write("/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/scores_pca_ukb.ht", overwrite=True)
related_samples_to_remove_ref.write("/mnt/grid/janowitz/home/references/1k_hgdp/related_remove_ref.ht",overwrite=True)


Run on Jupyter

In [ ]:
import pandas as pd

scores_pca_ref = hl.read_table("/mnt/grid/janowitz/home/references/1k_hgdp/scores_pca_ref.ht")
scores_pca_ukb = hl.read_table("/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/scores_pca_ukb.ht")
this_ref = hl.read_matrix_table('/mnt/grid/janowitz/home/references/1k_hgdp/ref_intersect.mt')

merge = scores_pca_ref.union(scores_pca_ukb)

merge = merge.annotate(
    training_pop=this_ref.cols()[merge.key].labeled_subpop)

recode = pd.read_excel('/mnt/grid/janowitz/home/references/1k_hgdp/recode.xlsx')
recode_ht = hl.Table.from_pandas(recode, key='labeled_subpop')

merge = merge.annotate(
    training_pop=recode_ht[merge.training_pop].superpop)

predictions_ref, classifer_rf_ref = assign_population_pcs(merge, pc_cols = merge.scores, known_col = 'training_pop', seed=501, min_prob = 0.70, missing_label='Other')

ukb_predictions = predictions_ref.semi_join(scores_pca_ukb) #Subset UKB samples

ukb_predictions.write("/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/ukb_predictions.ht", overwrite=True)

Summarize population calls

In [ ]:
ukb_predictions_pd = ukb_predictions.to_pandas()
ukb_predictions_pd = ukb_predictions_pd[["s", "pop"]]
ukb_predictions_pd = ukb_predictions_pd[ukb_predictions_pd['pop'] != "Other"]
ukb_predictions_pd['pop'].value_counts()

## Trigger finger GWAS

In [ ]:
import hail as hl
import os
import pandas as pd
import subprocess
from gnomad.sample_qc.ancestry import *

#Define memory and CPU availability

tmp = "/mnt/grid/janowitz/rdata_norepl/tmp"

os.environ["SPARK_LOCAL_DIRS"]=tmp
os.environ["PYSPARK_SUBMIT_ARGS"] ="--conf spark.network.timeout=15m --conf spark.executor.heartbeatInterval=10m --conf spark.memory.fraction=1.0 --driver-memory 2880G pyspark-shell"


hl.init(default_reference='GRCh37', master ='local[96]',min_block_size=128, local_tmpdir=tmp, tmp_dir=tmp)


#Pre-process ancestry predictions, merge with phenotype data
ukb_predictions = hl.read_table("/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/ukb_predictions.ht")

ukb_predictions_pd = ukb_predictions.to_pandas()
ukb_predictions_pd = ukb_predictions_pd[["s", "pop"]]
ukb_predictions_pd = ukb_predictions_pd[ukb_predictions_pd['pop'] != "Other"]
print(ukb_predictions_pd['pop'].value_counts(), flush=True)

phenotypes = pd.read_csv('/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/trigger_gwas.csv')
phenotypes = phenotypes.drop(phenotypes.columns[0], axis=1)
phenotypes = phenotypes.rename(columns={'id': 's'})
phenotypes['s'] = phenotypes.s.astype(int)
ukb_predictions_pd['s'] = ukb_predictions_pd.s.astype(int)

phenotypes = phenotypes.merge(ukb_predictions_pd, how='inner', on='s')

populations = phenotypes['pop'].unique().tolist()
#populations = ["CSA"]

#Add relationship matrix (UKB-provided KING analysis)
rel = hl.import_table('/mnt/grid/ukbiobank/data/ApplicationXXXXX/rawdata/ukbXXXXX_rel_s488250.dat', impute=True,delimiter='\s+', 
                     types={'ID1': hl.tstr, 'ID2': hl.tstr})
rel = rel.select(rel.ID1, rel.ID2, rel.Kinship)

#Import GRCh37 data (filtered to call rate, MAF, no H-W filtering at present)
ukb_gwas = hl.read_matrix_table('/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/ukb_grch37_filtered_forgwas.mt')
ukb_gwas = ukb_gwas.filter_rows(ukb_gwas.variant_qc.AC[1] > 100) #As per Regenie documentation
print(ukb_gwas.count(), flush=True)

#Prune GRCh37 data using SNP list (in_PCA) from marker paper
snp_ukb = hl.import_table('/mnt/grid/janowitz/home/skleeman/ukbiobank/cancergwas/relatedness/ukb_snp_qc.txt', impute=True, delimiter='\s+')
snp_ukb = snp_ukb.filter(snp_ukb.in_PCA==1)
snp_ukb = snp_ukb.key_by(snp_ukb.rs_id)
ukb_gwas_pruned = ukb_gwas.filter_rows(hl.is_defined(snp_ukb[ukb_gwas.rsid]))

for pop in populations:
    print(pop, flush=True)
    phenotypes['s'] = phenotypes.s.astype(str)
    
    #Subset covariate file, make suitable for BOLT-LMM
    
    subset = phenotypes[phenotypes['pop'] == pop]
    
    frame = { 'FID': subset['s'], 'IID': subset['s'], 'year_of_birth': subset['year_of_birth_f34_0_0'],
             'sex': subset['sex_f31_0_0'],'array': subset['genotype_measurement_batch_f22000_0_0'],
        'centre': subset['uk_biobank_assessment_centre_f54_0_0']}
    frame2 = { 'FID': subset['s'], 'IID': subset['s'], 'base_analysis':subset['base_analysis'], 'base_analysis_and_self':subset['base_analysis_and_self'], 'double_positive':subset['double_positive'],'base_without_CTS':subset['base_without_CTS'],'mixed_tf_cts':subset['mixed_tf_cts']}
    
    covariates = pd.DataFrame(frame) 
    phenotypes_out = pd.DataFrame(frame2) 
    print(phenotypes_out['trigger_scar'].value_counts(), flush=True)
    print(phenotypes_out['hypertrophic_scar'].value_counts(), flush=True)
    
    #Subset GRCH37 version of UK biobank dataset
    
    subset_ht = hl.Table.from_pandas(subset['s'].to_frame(), key='s') 
    ukb_pop_gwas = ukb_gwas.filter_cols(hl.is_defined(subset_ht[ukb_gwas.col_key]))
    ukb_pop_gwas = hl.variant_qc(ukb_pop_gwas)
    ukb_pop_gwas = ukb_pop_gwas.filter_rows(ukb_pop_gwas.variant_qc.AC[1] > 20) #As per Regenie documentation
    ukb_pop_gwas_pruned = ukb_gwas_pruned.filter_cols(hl.is_defined(subset_ht[ukb_gwas_pruned.col_key]))

    #Save filtered PLINK file (GRCH37, include rsid)
    
    folder = '/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/gwas_trigger/' + pop + '/filtered_grch37'
    
    if not os.path.exists(folder + '.bim'):
        hl.export_plink(ukb_pop_gwas, folder,
                    fam_id = ukb_pop_gwas.fam_id, ind_id = ukb_pop_gwas.s, pat_id = ukb_pop_gwas.pat_id, mat_id = ukb_pop_gwas.mat_id,
                    is_female = ukb_pop_gwas.is_female, varid = ukb_pop_gwas.rsid, pheno = -9)
    
    #Run PCA on all filtered variants, per population, removing relateds subjects per population
    
    rel_sub = rel.filter(hl.is_defined(subset_ht[rel.ID1]) & hl.is_defined(subset_ht[rel.ID2]))
    pairs = rel_sub.filter(rel_sub['Kinship'] > 0.0442) #Third degree relatives per KING documentation
    related_samples_to_remove = hl.maximal_independent_set(pairs.ID1, pairs.ID2, False)
    
    print("Related + ", pop, related_samples_to_remove.count(), flush=True)
    
    _, scores_pca_ref, _ = run_pca_with_relateds(ukb_pop_gwas_pruned, related_samples_to_remove, 
                                                               n_pcs=10, autosomes_only=True)
    
    scores_pca_ref = scores_pca_ref.transmute(**{f'PC{i}': scores_pca_ref.scores[i - 1] for i in range(1, 11)})
    scores_pd = scores_pca_ref.to_pandas()
    scores_pd = scores_pd.rename(columns={'s': 'IID'})
    
    #Add PCA results to covariates file, then save
    
    covariates = covariates.merge(scores_pd, left_on='IID', right_on='IID')
    folder = '/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/gwas_trigger/' + pop + '/covariates.tsv'
    covariates.to_csv(folder, sep="\t", index=False)
    folder = '/mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/gwas_trigger/' + pop + '/phenotypes.tsv'
    phenotypes_out.to_csv(folder, sep="\t", index=False)

In [ ]:
#!/bin/bash

for folder in /mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/gwas_trigger/*/; do

base=$(basename $folder)

echo $base

cat <<EOF >./scripts/$base.sh

#!/bin/bash
#$ -cwd
#$ -pe threads 16
#$ -l m_mem_free=8G
#$ -N gwas_$base
#$ -o $base.txt
#$ -e $base.txt

cd /mnt/grid/ukbiobank/data/ApplicationXXXXX/skleeman/gwas_trigger/$base

regenie \
  --step 1 --threads 16 \
  --bed filtered_grch37 \
  --phenoFile phenotypes.tsv \
  --covarFile covariates.tsv \
  --bt \
  --bsize 1000 \
  --lowmem \
  --lowmem-prefix /mnt/grid/janowitz/rdata_norepl/tmp/regenie/$base \
  --out ukb_step1_BT

for chr in {1..22}
do

regenie \
  --step 2 --threads 16 \
  --bgen /mnt/grid/ukbiobank/data/ApplicationXXXXX/imputed_missing/ukb_imp_chr\${chr}_v3.bgen \
  --sample /mnt/grid/ukbiobank/data/ApplicationXXXXX/rawdata/ukbXXXXX_imp_chr1_v3_s487282.sample \
  --phenoFile ukb_phenotypes_BT.txt \
  --covarFile ukb_covariates.txt \
  --bt \
  --firth 0.01 --approx \
  --pred ukb_step1_BT_pred.list \
  --bsize 400 \
  --minINFO 0.8 \
  --split \
  --out ukb_step2_BT_chr\${chr}

done

EOF
done